In [29]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings(action = 'ignore')

In [4]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [5]:
cancer_data= load_breast_cancer()
X_data, y_label = cancer_data.data, cancer_data.target
X_train,X_test,y_train,y_test = train_test_split(X_data, y_label, test_size=0.2, random_state= 0)

In [6]:
#개별 ML모델 생성
knn_clf = KNeighborsClassifier(n_neighbors = 4)
rf_clf = RandomForestClassifier(n_estimators = 100, random_state =0)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators=100)
#스태킹으로 만들어진 데이터 세트를 학습, 예측할 최종 모델
lr_final = LogisticRegression(C=10)

In [7]:
knn_clf.fit(X_train ,y_train)
rf_clf.fit(X_train,y_train)
dt_clf.fit(X_train, y_train)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=100)

In [8]:
knn_pred = knn_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
dt_pred = dt_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)

In [20]:
target=[knn_pred, rf_pred, dt_pred, ada_pred]
for m in target:
    print(m.shape)

(114,)
(114,)
(114,)
(114,)


In [21]:
for pred,name in zip(target,['knn','rf','dt','ada']):
    print(f"{name}: {accuracy_score(y_test,pred)}\n")

knn: 0.9210526315789473

rf: 0.9649122807017544

dt: 0.9122807017543859

ada: 0.956140350877193



# 스태킹 앙상블

In [22]:
pred = np.array(target)

In [23]:
pred

array([[0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1,
        0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1,
        0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
        0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0,
        1, 0, 0, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1,
        1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1,
        0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
        0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
        1, 0, 0, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
        0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0,

In [24]:
pred.shape

(4, 114)

- transpose로 행과 열 위치 바꾸기

In [26]:
pred = pred.transpose()

In [27]:
pred.shape

(114, 4)

- 최종 메타 모델의 예측 정확도

In [36]:
lr_final.fit(X_train,y_train)

LogisticRegression(C=10)

In [37]:
accuracy_score(y_test,lr_final.predict(X_test))

0.9385964912280702

# CV 세트 기반의 스태킹

In [38]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

In [70]:
#개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds):
    
    #지정된 n_folds값으로 KFold 생성
    kf = KFold(n_splits =n_folds, shuffle=False, random_state=0)
    #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화
    train_fold_pred = np.zeros((X_train_n.shape[0],1))

    test_pred = np.zeros((X_test_n.shape[0],n_folds))
    #print(X_train_n)
    #print(kf.split(X_train_n))
    print(model.__class__.__name__, 'model 시작') #model 이름 추출 __class__.__name__
   
    for folder_counter, (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        #print(train_index, valid_index)
        #입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 세트 추출
        print('\t 폴드 세트: ', folder_counter,'시작')
        X_tr = X_train_n[train_index]
        y_tr = y_train_n[train_index]
        X_te = X_train_n[valid_index]
        
        #폴드 세트 내부에서 다시 만들어진 학습데이터로 모델 학습
        model.fit(X_tr, y_tr)
        
        #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        
        #입력된 원본 테스트 데이터를 폴드 세트 내 학습된 기반 모델에서 예측 후 데이터 저장
        test_pred[:, folder_counter] = model.predict(X_test_n) 
    
    #폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성
    test_pred_mean = np.mean(test_pred, axis = 1).reshape(-1,1)
    
    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred, test_pred_mean

In [72]:
knn_train, knn_test = get_stacking_base_datasets(knn_clf, X_train, y_train, X_test, 7)
rf_train, rf_test = get_stacking_base_datasets(rf_clf, X_train, y_train, X_test, 7)
dt_train, dt_test = get_stacking_base_datasets(dt_clf, X_train, y_train, X_test, 7)
ada_train, ada_test = get_stacking_base_datasets(ada_clf, X_train, y_train, X_test, 7)

KNeighborsClassifier model 시작
	 폴드 세트:  0 시작
	 폴드 세트:  1 시작
	 폴드 세트:  2 시작
	 폴드 세트:  3 시작
	 폴드 세트:  4 시작
	 폴드 세트:  5 시작
	 폴드 세트:  6 시작
RandomForestClassifier model 시작
	 폴드 세트:  0 시작
	 폴드 세트:  1 시작
	 폴드 세트:  2 시작
	 폴드 세트:  3 시작
	 폴드 세트:  4 시작
	 폴드 세트:  5 시작
	 폴드 세트:  6 시작
DecisionTreeClassifier model 시작
	 폴드 세트:  0 시작
	 폴드 세트:  1 시작
	 폴드 세트:  2 시작
	 폴드 세트:  3 시작
	 폴드 세트:  4 시작
	 폴드 세트:  5 시작
	 폴드 세트:  6 시작
AdaBoostClassifier model 시작
	 폴드 세트:  0 시작
	 폴드 세트:  1 시작
	 폴드 세트:  2 시작
	 폴드 세트:  3 시작
	 폴드 세트:  4 시작
	 폴드 세트:  5 시작
	 폴드 세트:  6 시작


In [73]:
Stack_final_X_train = np.concatenate((knn_train, rf_train, dt_train, ada_train),axis=1)
Stack_final_X_test = np.concatenate((knn_test, rf_test, dt_test, ada_test),axis=1)

In [74]:
print("원본 학습 피처 데이터 Shape: ", X_train.shape,'원본 테스트 피처 Shape: ', X_test.shape)
print('스태킹 학습 피처 데이터 Shape: ', Stack_final_X_train.shape, '스태킹 테스트 피처 데이터 Shape: ', Stack_final_X_test.shape)

원본 학습 피처 데이터 Shape:  (455, 30) 원본 테스트 피처 Shape:  (114, 30)
스태킹 학습 피처 데이터 Shape:  (455, 4) 스태킹 테스트 피처 데이터 Shape:  (114, 4)


In [76]:
lr_final.fit(Stack_final_X_train, y_train)

LogisticRegression(C=10)

In [80]:
stack_final = lr_final.predict(Stack_final_X_test)
print(f"최종 메타 모델의 예측 정확도: {np.round(accuracy_score(y_test,stack_final),4)}")

최종 메타 모델의 예측 정확도: 0.9737
